In [2]:
!pip3 install pyodbc


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pyodbc
conn=pyodbc.connect('drive={SQL Server};'
                    'Server=server_name;'
                    'Database=database_name;'
                    'Trusted_Connection=yes;')
cursor=conn.cursor()
cursor.execute('select * from database_name.table')

records=cursor.fetchall()

BILL PRINTING ACTIVITY

In [ ]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DIPTI\SQLEXPRESS;"
    "DATABASE=python;"
    "Trusted_Connection=yes"
)

cursor = conn.cursor()

#print('-'*10+'MENU'+'-'*10)
#cursor.execute('select ISNULL(max(orderid),0)+1 from OrderDetails')
#orderid=cursor.fetchone()[0]

# for the menu show
print('-'*60)
print(''*22+'MENU')
print('-'*60)

cursor.execute('select * from menu')
for row in cursor.fetchall():
    print(f"{row.menuid}. {row.name} - {row.price} ({row.category})")
    
tableno=int(input('\nEnter Table.No: '))
cursor.execute('select isnull(max(orderid),0)+1 from OrderDetails')
orderid=cursor.fetchone()[0]
bill_item=[]   # we add items in the list
grand_total=0

while True:
    
	menuid=int(input('Enter Menu id: '))
	quantity=int(input('Enter Quantity: '))

	cursor.execute('select name,price from menu where menuid=?',(menuid,))
	item=cursor.fetchone()

	if item:
		name = item.name
		price = item.price
		total = price * quantity
		grand_total+=total

		bill_item.append((name,total))

		cursor.execute("""insert into OrderDetails
				 (orderid,menuid,quantity,tableno,totalprice)
				 values (?,?,?,?,?)""",
				 (orderid,menuid,quantity,tableno,total))
		conn.commit()
		print('Item Added')
	else:
		print('Invalid menu id')

	more=input('Add more items? (Y/N)').lower()
	if more != 'y':
		break

print('\n'+'-'*50)
print(' '*18+"Dipti's Kitchen")
print('-'*50)
print(f'Table No: {tableno}')
print('-'*50)

for name,total in bill_item:
	print(f"{name:<20}{total:<10}")
print('-'*50)
print(f'Total Bill: {'':<11}{grand_total}')
print('-'*50)
cursor.close()   
conn.close() 
#cursor.close()   

------------------------------------------------------------
MENU
------------------------------------------------------------
1. Soup - 120.00 (Starter)
2. Spring Rolls - 150.00 (Starter)
3. Paneer Butter Masala - 220.00 (Main Course)
4. Fried Rice - 200.00 (Main Course)
5. Chicken Masala - 280.00 (Main Course)


<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\shind\AppData\Local\Temp\ipykernel_8756\964208803.py:5: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=DIPTI\SQLEXPRESS;"


Item Added
Item Added
Item Added

--------------------------------------------------
                  Dipti's Kitchen
--------------------------------------------------
Table No: 2
--------------------------------------------------
Soup                240.00    
Chicken Masala      280.00    
Spring Rolls        300.00    
--------------------------------------------------
Total Bill:            820.00
--------------------------------------------------


In [7]:
import pyodbc
import os
from datetime import datetime


class CreateBill:
     def __init__(s):
        s.conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=DIPTI\SQLEXPRESS;"
            "DATABASE=python;"
            "Trusted_Connection=yes;"
        )
        s.cursor = s.conn.cursor()
        s.bill_items = []   # we are adding here menuid, name, quantity, total
        s.grand_total = 0
        s.orderid = 0
        s.tableno = 0
        s.bill_time = datetime.now()
     def show_menu(s):
        print('-'*60)
        print('{:^60}'.format('MENU'))
        print('-'*60)

        s.cursor.execute('select * from menu')
        for row in s.cursor.fetchall():
            print(f"{row.menuid}. {row.name} - {row.price} ({row.category})")

     def take_order(s):
        s.tableno = int(input('Enter Table No: '))
        s.cursor.execute('select isnull(max(orderid),0)+1 from OrderDetails')
        s.orderid = s.cursor.fetchone()[0]

        while True:
            menuid = int(input('Enter Menu ID: '))
            quantity = int(input('Enter Quantity: '))
            s.cursor.execute(
                'select name,price from menu where menuid=?',
                (menuid,))
            item = s.cursor.fetchone()
            if item:
                name = item.name
                price = item.price
                total = price * quantity

                s.grand_total += total
                s.bill_items.append((menuid, name, quantity, total))

                s.cursor.execute("""
                    insert into OrderDetails
                    (orderid, menuid, quantity, tableno, totalprice)
                    values (?, ?, ?, ?, ?)
                """, (s.orderid, menuid, quantity, s.tableno, total))

                s.conn.commit()

                print("Menu added!!")

            else:
                print("Invalid Menu ID selected")

            more = input("Would like to add more items? (y/n): ").lower()
            if more != 'y':
                break
        s.remove_item()
     def remove_item(s):
        while True:
            if not s.bill_items:
                print("No items to remove.")
                return
            print("\nCurrent Cart:")
            for index, item in enumerate(s.bill_items, start=1):
                print(f"{index}. {item[1]} - Qty:{item[2]} - Total:{item[3]}")

            want_remove=input("Do you want to remove any item? (y/n): ").lower()

            if want_remove!='y':
                break
            remove_index = int(input("Enter item number to remove: "))
            if 1 <= remove_index <= len(s.bill_items):
                removed_item = s.bill_items.pop(remove_index - 1)
                s.grand_total -= removed_item[3]
                s.cursor.execute("""
                    delete from OrderDetails
                    where orderid=? and menuid=? and quantity=?
                """, (s.orderid, removed_item[0], removed_item[2]))
                s.conn.commit()
                print("Item removed successfully")
            else:
                print("Invalid selection")
     def print_bill(s):
        formatted_time = s.bill_time.strftime("%d-%m-%Y %H:%M:%S")

        print('\n' + '-'*60)
        print('{:^60}'.format("Dipti's Kitchen"))
        print('-'*60)
        print(f"Order ID : {s.orderid}")
        print(f"Table No : {s.tableno}")
        print(f"Date & Time : {formatted_time}")
        print('-'*60)

        print(f"{'Item Name':<25}{'Qty':>10}{'Total':>15}")
        print('-'*60)

        for item in s.bill_items:
            print(f"{item[1]:<25}{item[2]:>10}{item[3]:>15.2f}")

        print('-'*60)
        print(f"{'Total Bill':<25}{'':>10}{s.grand_total:>15.2f}")
        print('-'*60)

        s.save_bill(formatted_time)
     def save_bill(s, formatted_time):
        folder = r"C:\Users\shind\Desktop\Backup"
        if not os.path.exists(folder):
            os.makedirs(folder)
        filename = os.path.join(folder, f"bill_{s.orderid}.txt")
        with open(filename, "w") as file:
            file.write("-"*60 + "\n")
            file.write("{:^60}\n".format("Dipti's Kitchen"))
            file.write("-"*60 + "\n")
            file.write(f"Order ID : {s.orderid}\n")
            file.write(f"Table No : {s.tableno}\n")
            file.write(f"Date & Time : {formatted_time}\n")
            file.write("-"*60 + "\n")
            file.write(f"{'Item Name':<25}{'Qty':>10}{'Total':>15}\n")
            file.write("-"*60 + "\n")
            for item in s.bill_items:
                file.write(f"{item[1]:<25}{item[2]:>10}{item[3]:>15.2f}\n")
            file.write("-"*60 + "\n")
            file.write(f"{'Total Bill':<25}{'':>10}{s.grand_total:>15.2f}\n")
            file.write("-"*60 + "\n")

        print(f"\nBill saved successfully at:\n{filename}")
     def close(s):
        s.cursor.close()
        s.conn.close()


obj = CreateBill()
obj.show_menu()
obj.take_order()
obj.print_bill()
obj.close()


------------------------------------------------------------
                            MENU                            
------------------------------------------------------------
1. Soup - 120.00 (Starter)
2. Spring Rolls - 150.00 (Starter)
3. Paneer Butter Masala - 220.00 (Main Course)
4. Fried Rice - 200.00 (Main Course)
5. Chicken Masala - 280.00 (Main Course)


<>:10: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\S'
C:\Users\shind\AppData\Local\Temp\ipykernel_15796\2854238415.py:10: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=DIPTI\SQLEXPRESS;"


Menu added!!
Menu added!!
Menu added!!

Current Cart:
1. Paneer Butter Masala - Qty:2 - Total:440.00
2. Fried Rice - Qty:2 - Total:400.00
3. Chicken Masala - Qty:3 - Total:840.00

------------------------------------------------------------
                      Dipti's Kitchen                       
------------------------------------------------------------
Order ID : 12
Table No : 2
Date & Time : 18-02-2026 00:25:55
------------------------------------------------------------
Item Name                       Qty          Total
------------------------------------------------------------
Paneer Butter Masala              2         440.00
Fried Rice                        2         400.00
Chicken Masala                    3         840.00
------------------------------------------------------------
Total Bill                                 1680.00
------------------------------------------------------------

Bill saved successfully at:
C:\Users\shind\Desktop\Backup\bill_12.txt
